In [97]:
# Import all the liberaries 
import torch 
import torch.nn as nn 
import pandas as pd
from torch.utils.data import DataLoader , Dataset

In [98]:
# Import minist dataset
df = pd.read_csv("Dataset/Mnist/train.csv")
df = df.iloc[:100000]
df.sample(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
36232,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39089,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31862,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3496,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18805,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
# train set split
import numpy as np
from sklearn.model_selection import train_test_split

y = df["label"]
x = df.drop(columns= ["label"])
x_train , x_test , y_train , y_test  = train_test_split(x , y , test_size= 0.2 , random_state= 42)
x_train.shape , y_train.shape

x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


In [100]:
# Create model class
import torch.optim as opt

class NeuralNetwork(nn.Module):
    def __init__(self , feature_count):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(feature_count , 256 , bias= True),
            nn.ReLU(),
            nn.Linear(256 , 128 , bias= True),
            nn.ReLU(),
            nn.Linear(128 , 64 ),
            nn.ReLU(),
            nn.Linear(64 , 10),
        )
        
    def forward(self , features):
        out = self.network(features)
        return out
    


In [101]:
# Create Dataset class
class CustomeDataset(Dataset):
    def __init__(self, featues , lables):
        
        self.features = featues.to(dtype = torch.float32)
        self.features = self.features/255
        self.labels = lables.to(dtype = torch.long)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self , index):
        return self.features[index] , self.labels[index]


In [102]:
# Declare hyperparamater 
batch_size = 16
lr = 0.01
epochs = 20

In [103]:
# Init model and Dataloder 
dataset_train = CustomeDataset(x_train , y_train)
dataloder_train = DataLoader(dataset_train , shuffle= True , batch_size= batch_size)

# Model
model = NeuralNetwork(x_train.shape[1])

loss_function = nn.CrossEntropyLoss()
opti = opt.Adam(model.parameters() , lr = lr)

In [104]:
x_train.shape

torch.Size([33600, 784])

In [105]:
from torchinfo import summary
summary(model , input_size=(33600, 784))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [33600, 10]               --
├─Sequential: 1-1                        [33600, 10]               --
│    └─Linear: 2-1                       [33600, 256]              200,960
│    └─ReLU: 2-2                         [33600, 256]              --
│    └─Linear: 2-3                       [33600, 128]              32,896
│    └─ReLU: 2-4                         [33600, 128]              --
│    └─Linear: 2-5                       [33600, 64]               8,256
│    └─ReLU: 2-6                         [33600, 64]               --
│    └─Linear: 2-7                       [33600, 10]               650
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 8.16
Input size (MB): 105.37
Forward/backward pass size (MB): 123.11
Params size (MB): 0.97
Estimated Total Size (MB): 229.45

In [106]:
# traing Loop

for epoch in range(epochs):
    total_epoch_loss = 0
    
    for train_features , train_lables in dataloder_train:
        
        # forward pass
        y_pred = model(train_features)
        
        # Loss calculation
        loss = loss_function(y_pred , train_lables)
        
        # clear gradient 
        opti.zero_grad()
         
        # backpropgation
        loss.backward()
        
        # paramter updates
        opti.step()
        
        total_epoch_loss += loss.item()
    
    print(f"Epoch : {epoch+1} , Loss : {total_epoch_loss/len(dataloder_train)}")
    

Epoch : 1 , Loss : 0.42172959933750376
Epoch : 2 , Loss : 0.2580945057279141
Epoch : 3 , Loss : 0.23234955482228542
Epoch : 4 , Loss : 0.2303788190496319
Epoch : 5 , Loss : 0.20524552047302894
Epoch : 6 , Loss : 0.21665942949942507
Epoch : 7 , Loss : 0.18366902628822918
Epoch : 8 , Loss : 0.158928002414934
Epoch : 9 , Loss : 0.17274977109929576
Epoch : 10 , Loss : 0.1635175941811082
Epoch : 11 , Loss : 0.13351602597082332
Epoch : 12 , Loss : 0.1533991668834941
Epoch : 13 , Loss : 0.1337636177155017
Epoch : 14 , Loss : 0.13254632868980215
Epoch : 15 , Loss : 0.1768407196108593
Epoch : 16 , Loss : 0.1365880494479851
Epoch : 17 , Loss : 0.11151483910812694
Epoch : 18 , Loss : 0.12847373132121712
Epoch : 19 , Loss : 0.11577489755458117
Epoch : 20 , Loss : 0.11326669160818166


In [107]:
model.eval() # for diff behavior in train and eval, like drop out or batchnor


NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [143]:
# Evalution 
test_dataset = CustomeDataset(x_test , y_test)
test_dataloder = DataLoader(test_dataset , batch_size= 16 , shuffle= False)

total = 0
correct = 0

with torch.no_grad():
    for test_feat , test_label in test_dataloder:
        
        output = model(test_feat)
        
        _ , pos = torch.max(output , dim= 1)
        
        total += test_label.shape[0]
        
        correct += (pos == test_label).sum().item()
    
    print(f"Accuracy : {correct/total}")
        
        

Accuracy : 0.9541666666666667


In [141]:
test_dataset[29][1]

tensor(3)